In [3]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from time import time

from numpy import (
    dot, float32 as REAL
)
import numpy as np
from gensim import matutils

Cargar Modelos originales

In [2]:
modeloD2V = Doc2Vec.load("./Modelos/modelo2_FINALD2V.model")
modeloW2V = Word2Vec.load("./Modelos/modelo10FINAL.model")

Obtener Vectores e inferencia

In [ ]:
#GUARDAR MODELO SIN INFERENCIA
modeloD2V.delete_temporary_training_data(keep_doctags_vectors=False, keep_inference=False)
modeloD2V.save('modelofree.model')
modeloD2V.trainables.save('neural.mm')


# Inferir Vectores

In [ ]:
#Vectores y matriz de inferencia
red = modeloD2V.trainables.save('neural.mm')
vectorlockf = red.vectors_lockf
syn1neg = red.syn1neg
del(red)

In [ ]:
modelo.trainables.vectors_lockf = vectorlockf
modelo.trainables.syn1neg = syn1neg
del(vectorlockf)
del(syn1neg)

In [ ]:
modelo.infer_vector(['hola'])

# Función similiar w2v

In [ ]:
keyedvectors = modeloW2V.wv
#keyedvectors.save('vectors.kv')
del(modeloW2V)

##### modelo original

In [ ]:
words = ['gestor', 'conocimiento']
#Modelo W2v
similar_words = keyedvectors.most_similar(words, topn=10)
similar_words

##### Pesos a las palabras y encontrar la media

In [29]:
from numpy import (
    dot, float32 as REAL
)
import numpy as np

from gensim import matutils

mean = [keyedvectors.get_vector('conocimiento')*1.0,keyedvectors.get_vector('gestor')*1.0]
array = np.array(mean)
#mean = matutils.unitvec(array.mean(axis=0)).astype(REAL)
array.mean(axis=0)

NameError: name 'keyedvectors' is not defined

##### Vectores normalizados

In [ ]:
from numpy import linalg as LA
norms=[]
for vector in keyedvectors.vectors:
    norms.append(LA.norm(vector))

##### Producto punto y normalizar 
Ordenar de mayor a menor y su cantidad

In [ ]:
dist = np.dot(keyedvectors.vectors[0:None], mean) / norms[0:None]
best = matutils.argsort(dist, topn=10 + 2, reverse=True) 
best

##### Encontrar la palabra

In [ ]:
keyedvectors.index2word[4933]

### Función Similar W2V old versión

In [3]:
modeloW2V.wv.init_sims()

In [4]:
#indice Enumerate
#print(modeloW2V.wv.vectors) Vector
print(modeloW2V.wv.index2word[1]) # palabra
#from numpy import linalg as 
#LA.norm(vector) Norm Vector

nariño


In [45]:
array = [modeloW2V.wv.vectors_norm[1]*1.0, modeloW2V.wv.vectors_norm[4]*1.0]
array1 = array[0]
array_mean=np.array(array).mean(axis=0)

In [120]:
final_mean = []
for vector in array:
    final_mean.append(sum(vector.tolist()) / len(vector))
final_mean

[-0.004976595592840264, -0.00048409564726171084]

In [10]:
def mean(a):
    return sum(a) / len(a)

mean_list = np.array(list(map(mean, zip(*array)))).astype('float32')

In [13]:
with open('vector.txt', 'w') as f:
    for item in mean_list.tolist():
        f.write("%s," % item)

In [113]:
comparison = mean_list == array_mean
equal_arrays = comparison.all() 
  
print(equal_arrays) 

True


In [46]:
from numpy import (
    dot, float32 as REAL
)
import numpy as np
from gensim import matutils

mean = matutils.unitvec(np.array(array).mean(axis=0)).astype(REAL)
limited = modeloW2V.wv.vectors_norm
dist = np.dot(limited, mean)

In [15]:
with open('vector.txt', 'w') as f:
    for item in mean.tolist():
        f.write("%s," % item)

In [152]:
best = matutils.argsort(dist, topn=10, reverse=True)
best

array([ 20,  64,  23, 437, 416, 215, 267,  10, 249, 336])

In [71]:
#Mean_list
best = matutils.argsort(dist, topn=10, reverse=True)
best

array([    4,     1,     5,    12,  7443,  6778, 10108,  6038,  9774,
        9615])

# Funcion similar d2v

In [61]:
len(modeloD2V.docvecs.vectors_docs_norm[1])

300

In [140]:
infer = modeloD2V.infer_vector(['conocimiento'])
modeloD2V.docvecs.most_similar([infer], topn=10)

[(215, 0.6421315670013428),
 (267, 0.6080042123794556),
 (10, 0.6039739847183228),
 (271, 0.6016193628311157),
 (416, 0.5938380360603333),
 (437, 0.5845867395401001),
 (295, 0.5813186168670654),
 (375, 0.5812559127807617),
 (39, 0.5790205001831055),
 (19, 0.5768672823905945)]

In [99]:
from numpy import (
    dot, float32 as REAL
)
import numpy as np

from gensim import matutils

mean = [infer*1.0]
array = np.array(mean)
mean = matutils.unitvec(array.mean(axis=0)).astype(REAL)
#array.mean(axis=0)

##### Producto punto y normalizar 
Ordenar de mayor a menor y su cantidad

In [101]:
dist = np.dot(modeloD2V.docvecs.vectors_docs_norm[0:None], mean)
best = matutils.argsort(dist, topn=10 + 2, reverse=True) 
best

array([ 20,  64,  23, 437, 416, 215, 267,  10, 249, 336, 309, 444])

##### Encontrar la palabra

In [ ]:
keyedvectors.index2word[4933]

# Vectores a ElasticSearch

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch()
es.ping()

True

**creación de indices Palabras**

In [9]:
mapping_palabras="""
{
    "mappings":{
        "properties":{
            "index":{"type": "integer"},
            "palabra":{"type": "keyword"},
            "vector":{
                "type": "dense_vector",
                "dims": 300
            },
            "norm_vec":{
                "type": "dense_vector",
                "dims": 300
            }
        }
    }
}
"""
# es.indices.delete(index='palabras')
es.indices.create(body=mapping_palabras, index='palabras')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'palabras'}

**creación de indices Documentos**

In [67]:
mapping_docs="""
{
    "mappings":{
        "properties":{
            "tag":{"type": "integer"},
            "vector":{
                "type": "dense_vector",
                "dims": 300
            },
            "norm_vec":{
                "type": "dense_vector",
                "dims": 300
            }
        }
    }
}
"""
# es.indices.delete(index='documentos')
es.indices.create(body=mapping_docs, index='documentos')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'documentos'}

In [13]:
print(es.indices.get('palabras'), end='\n\n')
print(es.indices.get('documentos'))

{'palabras': {'aliases': {}, 'mappings': {'properties': {'index': {'type': 'integer'}, 'norm_vec': {'type': 'dense_vector', 'dims': 300}, 'palabra': {'type': 'keyword'}, 'vector': {'type': 'dense_vector', 'dims': 300}}}, 'settings': {'index': {'creation_date': '1608162403100', 'number_of_shards': '1', 'number_of_replicas': '1', 'uuid': 'v2mqowdGSuGXzRYCOihh3A', 'version': {'created': '7060099'}, 'provided_name': 'palabras'}}}}

{'documentos': {'aliases': {}, 'mappings': {'properties': {'norm_vec': {'type': 'dense_vector', 'dims': 300}, 'tag': {'type': 'integer'}, 'vector': {'type': 'dense_vector', 'dims': 300}}}, 'settings': {'index': {'creation_date': '1607123586198', 'number_of_shards': '1', 'number_of_replicas': '1', 'uuid': 'hFkHM_O3TWmms3zkhR36BQ', 'version': {'created': '7060099'}, 'provided_name': 'documentos'}}}}


**Datos para Palabras**

In [25]:
type(modeloW2V.wv.vocab['diagnostico'].index)

int

In [28]:
# for i, (vector, vector_norm, palabra) in enumerate(zip(modeloW2V.wv.vectors, modeloW2V.wv.vectors_norm, modeloW2V.wv.vocab.keys())):

#     data = {
#         "index":i,
#         "palabra":palabra,
#         "vector":vector,
#         "norm_vec":vector_norm
#     }

#     es.index(index="palabras", body=data, id=i)

# print(es.count(index='palabras'))

for key, value in modeloW2V.wv.vocab.items():
    
    index = value.index
    vector = modeloW2V.wv.vectors[index]
    vector_norm = modeloW2V.wv.vectors_norm[index]
    data = {
        "doc":{
            "index":index,
            "palabra":key,
            "vector":vector,
            "norm_vec":vector_norm
        }
    }
    
    es.index(index="palabras", body=data, id=index)
    es.update(index="palabras", body=data, id=index)
    
    
print(es.count(index='palabras'))


{'count': 11445, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


**Datos Documentos D2V**

In [44]:
for i, (vector, vector_norm) in enumerate(zip(modeloD2V.docvecs.vectors_docs, modeloD2V.docvecs.vectors_docs_norm)):

    data = {
        "tag":i+1,
        "vector":vector,
        "norm_vec":vector_norm
    }

    es.index(index="documentos", body=data, id=i+1)

print(es.count(index='documentos'))

{'count': 451, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


# Calculando algoritmos desde Elastic

In [10]:
palabra = es.get(index='palabras', id=1)['_source']
type(palabra['index'])

int

### W2V Most Similar

Consultamos los vectores correspondientes a las palabras a buscar

In [47]:
query_vecs = {
    "query":{
        "bool":{
            "should":[
                {"term": { "palabra": "ontologias" }}
            ]
        }
    },
    "_source": ["palabra", "norm_vec"]
}
vectores = es.search(index='palabras', body=query_vecs)['hits']['hits']

Calculamos la Media de los vectores y transformamos a vector unitario

In [48]:
norm_vecs = [vector['_source']['norm_vec'] for vector in vectores]
vector_mean = np.array(norm_vecs).mean(axis=0)
unit_vec = vector_mean / np.linalg.norm(vector_mean)

Calculamos producto Punto en ElasticSearch y miramos el puntaje

In [49]:
query_sims = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec
        }
      }
    }
  },
  "size": 11,
  "_source": ["palabra", "index"]
}
most_similar = es.search(index='palabras', body=query_sims)['hits']['hits']
most_similar = [(palabra['_source']['index'],palabra['_source']['palabra'], palabra['_score']) for palabra in most_similar]
most_similar

[(2202, 'ontologias', 1.0),
 (4918, 'gestor', 0.85891134),
 (6645, 'sparql', 0.8413498),
 (6680, 'umayux', 0.8379714),
 (11329, 'buscador', 0.8294513),
 (4933, 'debilmente', 0.82685596),
 (11195, 'solicitada', 0.82475597),
 (6646, 'rdf', 0.8210847),
 (11316, 'evacuacion', 0.81534725),
 (11379, 'maskana', 0.8088352),
 (6643, 'motor', 0.80510694)]

In [50]:
words = ['ontologias']
similar_words = modeloW2V.wv.most_similar(words, topn=10)
similar_words

[('gestor', 0.8589113354682922),
 ('sparql', 0.8413498997688293),
 ('umayux', 0.8379713296890259),
 ('buscador', 0.829451322555542),
 ('debilmente', 0.8268558979034424),
 ('solicitada', 0.8247559666633606),
 ('rdf', 0.82108473777771),
 ('evacuacion', 0.8153471946716309),
 ('maskana', 0.8088351488113403),
 ('motor', 0.8051068782806396)]

### D2V Most Similar

Inferimos el Vector y calculamos la Media, luego se transforma  vector unitario

In [52]:
infer_vec = [modeloD2V.infer_vector(['medicina'])]
vec_mean_doc = np.array(infer_vec).mean(axis=0)
unit_vec_doc = vec_mean_doc / np.linalg.norm(vec_mean_doc)

Realizamos la consulta a Elastic con producto Punto

In [55]:
query_sims_docs = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec_doc
        }
      }
    }
  },
  "size": 10,
  "_source": ["tag"]
}

most_similar_docs = es.search(index='documentos', body=query_sims_docs)['hits']['hits']
most_similar_docs = [(doc['_source']['tag'], doc['_score']) for doc in most_similar_docs]
# most_similar_docs = [doc['_source']['tag'] for doc in most_similar]
most_similar_docs

[(371, 0.60009134),
 (93, 0.5903249),
 (5, 0.58496916),
 (56, 0.57999855),
 (244, 0.57189333),
 (96, 0.56820834),
 (405, 0.56594735),
 (440, 0.56486785),
 (315, 0.56443554),
 (60, 0.5625713)]

In [67]:
infer = modeloD2V.infer_vector(['medicina'])
sims_docs = modeloD2V.docvecs.most_similar([infer], topn=10)
for doc in sims_docs:
    print(f'({doc[0]+1}, {doc[1]})')

(371, 0.6050417423248291)
(93, 0.5903229713439941)
(56, 0.5860195159912109)
(139, 0.5759848356246948)
(440, 0.5720837116241455)
(5, 0.5701383352279663)
(405, 0.5683046579360962)
(381, 0.565326988697052)
(60, 0.5643856525421143)
(244, 0.5641034841537476)
